# You Do 2

Importing libraries and reading movielens dataset

In [1]:
import numpy as np
import pandas as pd

df = pd.read_csv('https://files.grouplens.org/datasets/movielens/ml-100k/u.data', delimiter=r'\t',engine='python',
names=['user_id', 'item_id', 'rating', 'timestamp'])
R = df.pivot(index='user_id', columns='item_id', values='rating').values

In [2]:
R

array([[ 5.,  3.,  4., ..., nan, nan, nan],
       [ 4., nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [ 5., nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan,  5., nan, ..., nan, nan, nan]])

In [3]:
# I need indexes of non-zero elements to train and score on.
irow, jcol = np.where(~np.isnan(R))

Train, validation and test split. I used validation set to optimize lambda hyperparameter

In [4]:
def split_data(R,irow,jcol):
    idx_all = np.random.choice(np.arange(100_000), 2000, replace=False)
    idx = np.random.choice(np.arange(2000), 1000, replace=False)
    val_idx = [val_id for val_id in idx_all if val_id not in idx]
    test_irow = irow[idx]
    test_jcol = jcol[idx]
    val_irow = irow[val_idx]
    val_jcol = jcol[val_idx]
    R_copy = R.copy()
    R_copy[test_irow, test_jcol] = np.nan
    R_copy[val_irow, val_jcol] = np.nan
    R_test_mask = R.copy() # Will be used only dor prediction
    R_test_mask[test_irow, test_jcol] = np.nan
    return R_copy, test_irow, test_jcol, val_irow, val_jcol, R_test_mask

In [5]:
# Loss function
def loss(b_user, b_item, R, irow, jcol):
    loss = 0
    for i, j in zip(irow, jcol):
        if np.isnan(R[i, j]):
            continue
        loss += (R[i, j] - b_user[i] - b_item[j]) ** 2 * 0.5
    return loss

In [6]:
# Gradient of loss function
def gradient(b_user, b_item, R, irow, jcol):
    b_user_grad = np.zeros(b_user.shape)
    b_item_grad = np.zeros(b_item.shape)
    for i, j in zip(irow, jcol):
        if np.isnan(R[i, j]):
            continue
        b_user_grad[i] += (R[i, j] - b_user[i] - b_item[j])
        b_item_grad[j] += (R[i, j] - b_user[i] - b_item[j])

    return b_user_grad, b_item_grad

In [7]:
# Gradient descent function
def gradient_descent(R, b_user, b_item, irow, jcol, lr, epochs):
    for epoch in range(epochs):
        b_user_grad, b_item_grad = gradient(b_user, b_item, R, irow, jcol)
        prev_b_user = b_user.copy()
        prev_b_item = b_item.copy()
        b_user += lr * b_user_grad 
        b_item += lr * b_item_grad
        if epoch % 10 == 0:
            print('loss:', loss(b_user, b_item, R, irow, jcol))
        # early stopping
        if np.linalg.norm(b_user - prev_b_user) < 1e-2 and np.linalg.norm(b_item - prev_b_item) < 1e-2:
            break

    return b_user, b_item

In [8]:
# Split data into training, validation and test sets
R_copy, test_irow, test_jcol, val_irow, val_jcol, R_test_mask = split_data(R,irow,jcol)

In [9]:
# Initialize biases
b_user = np.random.randn(943)
b_item = np.random.randn(1682)

In [10]:
# Gradient descent
b_user, b_item = gradient_descent(R_copy, b_user, b_item, irow, jcol, 0.001, 1000)

loss: 377212.85336741013
loss: 76661.6869947853
loss: 57366.645335132285
loss: 50445.88796956057
loss: 47078.01069912948
loss: 45177.82804883845
loss: 44000.73670352827
loss: 43221.76101390269
loss: 42679.889742517276
loss: 42287.79542083556
loss: 41994.768776683224
loss: 41769.754624955014
loss: 41592.906266461214
loss: 41451.0879914769
loss: 41335.34530124266
loss: 41239.41648494469
loss: 41158.823541521975
loss: 41090.29918488571
loss: 41031.41581630894
loss: 40980.3395703257
loss: 40935.66384921904
loss: 40896.29453353066
loss: 40861.369470052894
loss: 40830.20111180686
loss: 40802.23505538483
loss: 40777.01966112312
loss: 40754.183509216055
loss: 40733.41846957244
loss: 40714.46684344235
loss: 40697.11149313042
loss: 40681.16818888507
loss: 40666.47961830014
loss: 40652.91065480279
loss: 40640.34458879892
loss: 40628.68010146801
loss: 40617.82881643971
loss: 40607.71330482417
loss: 40598.265448644815
loss: 40589.425089748926
loss: 40581.138907667155
loss: 40573.35948234932
loss: 4

In [11]:
# Calculate RMSE
def rmse(R, b_user, b_item, irow, jcol):
    return np.sqrt(2 * loss(b_user, b_item, R, irow, jcol) / len(irow))

In [12]:
# Calculate RMSE on test set
print('Non regularized loss function model RMSE:' , rmse(R, b_user, b_item, test_irow, test_jcol))

Non regularized loss function model RMSE: 0.9681046554908976


## Regularized Loss Function Steps

In [13]:
# Regularized loss function
def loss_reg(b_user, b_item, R, irow, jcol, lmbda):
    loss = 0
    for i, j in zip(irow, jcol):
        if np.isnan(R[i, j]):
            continue
        loss += (R[i, j] - b_user[i] - b_item[j]) ** 2 * 0.5 + lmbda/2 * (b_user[i] ** 2 + b_item[j] ** 2)
    return loss

In [14]:
# Regularized gradient of loss function
def gradient_reg(b_user, b_item, R, irow, jcol, lmbda):
    b_user_grad = np.zeros(b_user.shape)
    b_item_grad = np.zeros(b_item.shape)
    for i, j in zip(irow, jcol):
        if np.isnan(R[i, j]):
            continue
        b_user_grad[i] += (R[i, j] - b_user[i] - b_item[j]) - lmbda * b_user[i]
        b_item_grad[j] += (R[i, j] - b_user[i] - b_item[j]) - lmbda * b_item[j]

    return b_user_grad, b_item_grad

In [15]:
# Regularized gradient descent function
def gradient_descent_reg(R, b_user, b_item, irow, jcol, lmbda, lr, epochs):
    for epoch in range(epochs):
        b_user_grad, b_item_grad = gradient_reg(b_user, b_item, R, irow, jcol, lmbda)
        prev_b_user = b_user.copy()
        prev_b_item = b_item.copy()
        b_user += lr * b_user_grad
        b_item += lr * b_item_grad
        if epoch % 10 == 0:
            print('loss:', loss_reg(b_user, b_item, R, irow, jcol, lmbda))
        # early stopping
        if np.linalg.norm(b_user - prev_b_user) < 1e-2 and np.linalg.norm(b_item - prev_b_item) < 1e-2:
            break

    return b_user, b_item

In [16]:
# Initialize biases
b_user = np.random.randn(943)
b_item = np.random.randn(1682)

In [17]:
# Hyperparameter optimization to find best lambda on validation set
best_rmse = float('inf')
best_lmbda = 0
for lmbda in [0.001, 0.01, 0.1, 1]:
    b_user, b_item = gradient_descent_reg(R_copy, b_user, b_item, irow, jcol, lmbda, 0.0005, 250)
    rmse_val = rmse(R, b_user, b_item, val_irow, val_jcol)
    if rmse_val < best_rmse:
        best_rmse = rmse_val
        best_lmbda = lmbda

loss: 557293.2071953361
loss: 119586.96863083639
loss: 80159.24880925246
loss: 65938.9770708831
loss: 58576.40308846983
loss: 54129.40784699711
loss: 51194.895939773305
loss: 49142.24260557388
loss: 47645.090662013936
loss: 46517.769799933114
loss: 45647.21740184988
loss: 44960.94205268114
loss: 44410.528771477344
loss: 43962.54689256373
loss: 43593.26208604325
loss: 43285.423208603665
loss: 43026.23667877849
loss: 42806.04847098954
loss: 42617.46259703246
loss: 42454.73709677593
loss: 42313.361250597685
loss: 42189.753994110106
loss: 42081.04514479395
loss: 41984.914305713464
loss: 41899.47063881205
loss: 44725.88352299339
loss: 44651.79789296341
loss: 44591.85018016707
loss: 44537.92343478617
loss: 44489.02461967675
loss: 44444.495785692474
loss: 44403.80383267787
loss: 44366.49995398292
loss: 44332.20103662535
loss: 44300.57756138863
loss: 44271.344571003596
loss: 44244.25453993302
loss: 44219.091598715095
loss: 44195.666790302865
loss: 44173.81413969138
loss: 44153.38737657552
loss

In [18]:
best_lmbda

0.01

In [19]:
# Regularized gradient descent with best lambda
b_user, b_item = gradient_descent_reg(R_test_mask, b_user, b_item, irow, jcol, best_lmbda, 0.0005, 1000)

loss: 93158.41211999886
loss: 50743.181315307665
loss: 47762.760693486816
loss: 46696.94272497925
loss: 46127.17723207204
loss: 45771.600545916444
loss: 45530.3739782717
loss: 45357.8020473515
loss: 45229.6200017272
loss: 45131.65446872691
loss: 45055.06260966397
loss: 44994.051758474714
loss: 44944.680107012246
loss: 44904.18114891234
loss: 44870.56263349545
loss: 44842.35827430468
loss: 44818.46857313212
loss: 44798.0556856148
loss: 44780.4721520854
loss: 44765.21146463626
loss: 44751.87307474996
loss: 44740.13716948011
loss: 44729.74619214669
loss: 44720.49110651175
loss: 44712.20105344817
loss: 44704.735470952204
loss: 44697.9780276167
loss: 44691.83190787644
loss: 44686.2161163274
loss: 44681.06255826405
loss: 44676.313716963
loss: 44671.920793706246
loss: 44667.84220939757
loss: 44664.04239078416
loss: 44660.49078214464
loss: 44657.161036703794
loss: 44654.030352074085
loss: 44651.078921779146
loss: 44648.28948074222
loss: 44645.64692720777
loss: 44643.13800709817
loss: 44640.751

In [20]:
# Function for regularized RMSE on test set with best model
def rmse_reg(R, b_user, b_item, irow, jcol, lmbda):
    return np.sqrt(2 * loss_reg(b_user, b_item, R, irow, jcol, lmbda) / len(irow))

In [21]:
# Calculate RMSE on test set
print('Regularized RMSE:' , rmse_reg(R, b_user, b_item, test_irow, test_jcol, best_lmbda))

Regularized RMSE: 1.0033994861270819
